# Programmation dynamique et mémoïsation

# 1. Problème du sujet 0 : 
Trouver la somme maximum qu'on peut atteindre en traversant les cellules d'un tableau d'entiers à 2 dimensions en n'autorisant que des déplacements vers la droite ou vers le bas. La cellule de départ est le coin supérieur gauche, et la cellule d'arrivée est le coin inférieur droit.

In [1]:
T = [[4, 1, 1, 3],
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

Parcours idéal de l'exemple : 4 - 2 - 3 - 1 - 5 - 1 : somme = 16

## Solution récursive :
Solution récurisve naive telle que proposée dans le sujet 0 de NSI.

In [ ]:
def somme_max(T, i, j):
    if i == j == 0:
        return T[i][j]
    elif i == 0:
        return T[0][j] + somme_max(T, 0, j-1)
    elif j == 0:
        return T[i][0] + somme_max(T, i-1, 0)
    else:
        return T[i][j] + max(somme_max(T, i-1, j), somme_max(T, i, j-1))

In [ ]:
somme_max(T, 2, 3)

**Problème** : on *re-calcule beaucoup de choses* ! **Très lourd en mémoire et la pile de récursion risque vite d'exploser**.

## Approche récursive avec mémoïsation (bottom-down)
L'idée est de **stocker dans un tableau annexe toutes les valeurs déjà calculées** une fois pour ne pas recommencer !

**On conserve l'approche descendante de la récursivité** : la solution globale fait appel au fur et à mesure au solutions des sous-problèmes (en vérifiant si une valeur n'a pas déjà été mémorisée avant de se lancer dans le calcul récursif).

In [4]:
def somme_max_memo(T, i, j):
    def sm(T, i, j):
        if i == j == 0:
            pass
        elif i == 0:
            if memoire[0][j-1] is not None:
                memoire[0][j] = T[0][j] + memoire[0][j-1]
            else:
                memoire[0][j] = T[0][j] + sm(T, 0, j-1)
        elif j == 0:
            if memoire[i-1][0] is not None:
                memoire[i][0] = T[i][0] + memoire[i-1][0]
            else:
                memoire[i][0] = T[i][0] + sm(T, i-1, 0)
        else:
            gauche, haut = memoire[i][j-1], memoire[i-1][j]
            if gauche is not None and haut is not None:
                memoire[i][j] = T[i][j] + max(gauche, haut)
            elif gauche is not None:
                memoire[i][j] = T[i][j] + max(gauche, sm(T, i-1, j))
            elif haut is not None:
                memoire[i][j] = T[i][j] + max(sm(T, i, j-1), haut)
            else:
                memoire[i][j] = T[i][j] + max(sm(T, i, j-1), sm(T, i-1, j))
        return memoire[i][j]
    
    memoire = [[None for _ in range(len(T[0]))] for _ in range(len(T))]
    memoire[0][0] = T[0][0]
    return sm(T, i, j)

In [10]:
def somme_max_memo2(T, i, j):
    def sm2(T, i, j):
        if memoire[i][j] is not None:
            return memoire[i][j]
        if i == j == 0:
            return T[0][0]
        elif i == 0:
            memoire[0][j] = T[0][j] + sm2(T, 0, j-1)
        elif j == 0:
            memoire[i][0] = T[i][0] + sm2(T, i-1, 0)
        else:
            gauche, haut = sm2(T, i, j-1), sm2(T, i-1, j)
            memoire[i][j] = T[i][j] + max(gauche, haut)
        return memoire[i][j]
    
    memoire = [[None for _ in range(len(T[0]))] for _ in range(len(T))]
    return sm2(T, i, j)

In [11]:
somme_max_memo2(T, 2, 3)

16

## Approche par la programmation dynamique
L'**approche ascendante de la programmation dynamique** consiste à construire **de façon itérative** les **solutions des sous-problèmes** pour atteindre la solution globale qui combine ces sous-problèmes.

In [6]:
def somme_max_dyn(T, i, j):    
    memoire = [[T[i][j] for j in range(len(T[0]))] for i in range(len(T))]
    for j in range(1, len(T[0])):
        memoire[0][j] = T[0][j] + memoire[0][j-1]
    for i in range(1, len(T)):
        memoire[i][0] = T[i][0] + memoire[i-1][0]
    for i in range(1, len(T)):
        for j in range(1, len(T[0])):
            memoire[i][j] = T[i][j] + max(memoire[i-1][j], memoire[i][j-1])
    return memoire[i][j]

In [7]:
somme_max_dyn(T, 2, 3)

16

## Simplification possible : optimisation de la mémoire
Il est possible de ne conserver la mémoire que d'une ligne à la fois.

In [8]:
def somme_max_dyn_opt(T, i, j):    
    ligne = [T[0][j] for j in range(len(T[0]))]
    for j in range(1, len(T[0])):
        ligne[j] = T[0][j] + ligne[j-1]
    for i in range(1, len(T)):
        ligne[0] = ligne[0] + T[i][0]
        for j in range(1, len(T[0])):
            ligne[j] = T[i][j] + max(ligne[j-1], ligne[j])
    return ligne[-1]

In [9]:
somme_max_dyn_opt(T, 2, 3)

16

# Voir la durée pour un tableau plus grand.
On propose un tableau de 10 lignes et 20 colonnes, de nombre aléaoires entre 1 et 9.

In [10]:
from random import randint

In [11]:
i, j = 10, 20
T = [[randint(1, 9) for _ in range(j)] for _ in range(i)]

for i in range(len(T)):
    for j in range(len(T[0])):
        print(T[i][j], end = '  ')
    print()

3  1  7  1  7  3  5  3  1  5  1  9  4  4  6  7  5  2  1  5  
6  3  6  3  9  9  2  4  3  4  5  7  8  7  9  8  1  7  3  4  
9  7  5  6  7  5  6  2  4  3  9  1  8  1  4  8  1  2  3  3  
1  1  9  3  7  7  4  8  9  1  4  3  2  8  5  2  5  1  6  1  
7  8  7  6  6  8  9  1  5  2  5  7  3  8  7  9  5  8  4  6  
4  9  7  9  7  8  2  8  4  3  5  2  5  8  4  8  5  9  3  6  
5  9  6  6  9  5  4  5  1  9  6  6  9  6  6  2  3  3  2  3  
8  7  7  5  5  3  9  1  2  5  8  4  4  8  1  9  7  6  2  4  
5  4  9  1  2  1  2  7  5  7  4  2  8  2  7  5  9  3  4  7  
4  9  4  3  4  5  1  4  2  1  9  4  2  2  6  6  6  4  8  9  


In [12]:
somme_max_dyn(T, i, j)

191

In [13]:
somme_max_dyn_opt(T, i, j)

191

In [14]:
somme_max_memo(T, i, j)

191

In [15]:
somme_max(T, i, j) # on sent passer le temps...

191

# Complément : Algo glouton
On propose une solution par algorithme glouton : pas forcément optimale !
Le critère local d'optimisation retenu est de s'orienter vers la cellule la plus "grosse" à chaque carrefour.

In [16]:
def somme_max_glouton(T, i, j):
    s = T[0][0]
    m, n = 0, 0
    while (n != len(T) - 1) or (m != len(T[0]) - 1):
        if m == len(T[0]) - 1:
            n += 1
        elif n == len(T) - 1:
            m += 1
        elif T[n][m+1] > T[n+1][m]:
            m += 1
        else:
            n += 1
        s += T[n][m]
    return s

## Cas où l'algo glouton fonctionne (coup de bol !)

In [17]:
T = [[4, 1, 1, 3],
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

i, j = len(T), len(T[0])
print(somme_max_glouton(T, i, j))
assert  somme_max_glouton(T, i, j) == somme_max_dyn_opt(T, i, j)

16


## Cas où l'algo glouton ne fonctionne pas (ben c'est pas étonnant qu'il y ait des échecs...)

In [18]:
T = [[4, 1, 1000, 3], # on va "rater" le 1000
     [2, 0, 2, 1],
     [3, 1, 5, 1]]

i, j = len(T), len(T[0])
print(somme_max_glouton(T, i, j))
assert  somme_max_glouton(T, i, j) == somme_max_dyn_opt(T, i, j)

16


AssertionError: 

## Cas aléatoire avec un tableau assez grand :
On compare la solution de l'algo glouton avec un algorithme où on navigue aléateoirement dans le tableau pour vérifier si l'algo glouton tend plutôt à donner une solution "intéressante".

In [19]:
from random import choice
def pile_ou_face():
    """ Une fonction de pile ou face à moitié inutile mais qui m'amuse... """
    return choice(['Pile', 'Face'])

In [20]:
def somme_alea(T, i, j):
    s = T[0][0]
    m, n = 0, 0
    while (n != len(T) - 1) or (m != len(T[0]) - 1):
        if m == len(T[0]) - 1:
            n += 1
        elif n == len(T) - 1:
            m += 1
        else:
            if pile_ou_face() == 'Pile':
                n += 1
            else:
                m += 1
        s += T[n][m]
    return s

### Tableau de 1000 lignes et 1000 colonnes :
On constate que l'algo glouton apporte quand même un intérêt par raport à la marche aléatoire, mais il n'atteint pas la solution optimale.

In [21]:
i, j = 1000, 1000
T = [[randint(1, 9) for _ in range(j)] for _ in range(i)]
    
print(somme_alea(T, i, j))
print(somme_max_glouton(T, i, j))
print(somme_max_dyn_opt(T, i, j))

9918
12699
14374
